# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [3]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [4]:
earthquake_json['metadata']

{'generated': 1663311174000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-08-15&endtime=2022-09-14',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.13.6',
 'count': 9366}

Each element in the JSON array `features` is a row of data for our dataframe.

In [5]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [6]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.5,
  'place': None,
  'time': 1663113432735,
  'updated': 1663113561594,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak022brrm24p',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak022brrm24p&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 35,
  'net': 'ak',
  'code': '022brrm24p',
  'ids': ',ak022brrm24p,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.81,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.5 - '},
 'geometry': {'type': 'Point', 'coordinates': [-149.1863, 62.3644, 0]},
 'id': 'ak022brrm24p'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

- `earthquake_properties_data` is a list of dictionaries. 
- each dictionary is a row in the pandas dataframe

In [14]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.5,None,1663113432735,1663113561594,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak022brrm24p,",",ak,",",origin,phase-data,",NaN,NaN,0.81,NaN,ml,earthquake,M 1.5 -
1,2.6,off the coast of Oregon,1663113386780,1663121255672,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,0.0,1.0,NaN,...,",us7000i7sx,",",us,",",dyfi,origin,phase-data,",16.0,2.032,0.38,308.0,ml,earthquake,M 2.6 - off the coast of Oregon
2,4.7,"272 km WNW of Haveluloto, Tonga",1663113163451,1663114295040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000i7sw,",",us,",",origin,phase-data,",78.0,4.873,0.85,121.0,mb,earthquake,"M 4.7 - 272 km WNW of Haveluloto, Tonga"
3,1.5,Central Alaska,1663112995205,1663113105980,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak022brrkfog,",",ak,",",origin,phase-data,",NaN,NaN,0.60,NaN,ml,earthquake,M 1.5 - Central Alaska
4,0.7,"95 km WNW of Aleneva, Alaska",1663112885028,1663113043244,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak022brrk43a,",",ak,",",origin,phase-data,",NaN,NaN,0.37,NaN,ml,earthquake,"M 0.7 - 95 km WNW of Aleneva, Alaska"


### (Optional) Write Data to CSV

In [15]:
df.to_csv('earthquakes.csv', index=False)

In [12]:
earthquake_properties_data[0:2]

[{'mag': 1.5,
  'place': None,
  'time': 1663113432735,
  'updated': 1663113561594,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak022brrm24p',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak022brrm24p&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 35,
  'net': 'ak',
  'code': '022brrm24p',
  'ids': ',ak022brrm24p,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.81,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.5 - '},
 {'mag': 2.6,
  'place': 'off the coast of Oregon',
  'time': 1663113386780,
  'updated': 1663121255672,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000i7sx',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000i7sx&format=geojson',
  'felt': 0,
  'cdi': 1,
  'mmi': None,
  'alert': None,
  'status': 'r

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>